<a href="https://colab.research.google.com/github/HyVeel/eksploracja-danych/blob/main/zaj6/obsluga_roznych_formatow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import sqlite3
import pandas as pd
import openpyxl

## openpyxl = Narzędzie do obsługi plików Excel

- Operuje bezpośrednio na plikach .xlsx (format Excela)

- Skupia się na strukturze pliku: arkusze, komórki, formatowanie

- Dobre do: tworzenia raportów, formatowania, pracy z wieloma arkuszami

In [72]:
# Stworzenie pliku xlsx

wb = openpyxl.Workbook()

arkusz1 = wb.active
arkusz1.title = "Produkty"
arkusz1.append(["Nazwa", "Cena", "Ilość"])
arkusz1.append(["Laptop", 3000, 5])

arkusz2 = wb.create_sheet("Klienci")
arkusz2.append(["Imię", "Email"])
arkusz2.append(["Jan", "jan@example.com"])

wb.save("moj_plik.xlsx")

In [73]:
# Odczytanie pliku xlsx

# Otwórz plik
wb = openpyxl.load_workbook("moj_plik.xlsx")

# Lista arkuszy
print(wb.sheetnames)  # ['Produkty', 'Klienci']

# Czytaj dane z arkusza
arkusz = wb["Produkty"]

for wiersz in arkusz.values:
    print(wiersz)

['Produkty', 'Klienci']
('Nazwa', 'Cena', 'Ilość')
('Laptop', 3000, 5)


# Wczytywanie plików programu Microsoft Excel

Biblioteka **pandas** umożliwia wczytywanie tabelarycznych danych zapisanych w plikach programu Excel 2003 i jego nowszych wersjach.

Można to zrobić za pomocą klasy `pandas.ExcelFile` lub funkcji pandas.`read_excel`.

Wewnętrznie narzędzia te korzystają z dodatkowych pakietów
xlrd i openpyxl umożliwiających odczyt plików XLS i XLSX.

In [74]:
# W celu skorzystania z klasy pandas.ExcelFile należy utworzyć jej egzemplarz, przekazując do jej konstruktora ścieżkę pliku xls lub xlsx:
xlsx = pd.ExcelFile('moj_plik.xlsx')

In [75]:
# Za pomocą uzyskanego obiektu można wyświetlić listę arkuszy zawartych w pliku:
xlsx.sheet_names

['Produkty', 'Klienci']

In [76]:
df_excel = pd.read_excel("moj_plik.xlsx", sheet_name="Klienci")
df_excel.head()

,Imię,Email
0,Jan,jan@example.com


# Obsługa baz danych

Ładowanie danych z bazy SQL do obiektu DataFrame jest dość proste, a pakiet pandas jest wyposażony w funkcje upraszczające ten proces.

W poniższym przykładzie tworzymy bazę danych SQLite, korzystając z wbudowanego pythonowego sterownika sqlite3:

In [77]:
query = """
    CREATE TABLE test (
        a VARCHAR(20),
        b VARCHAR(20),
        c REAL,
        d INTEGER
    );
    """

In [78]:
conn = sqlite3.connect('mydata.sqlite')

In [79]:
conn.execute(query)

In [80]:
conn.commit()

In [81]:
# Teraz wprowadzimy do naszej bazy kilka wierszy danych:
data = [('Warszawa', 'Polska', 2, 6),
          ('Berlin', 'Niemcy',4, 3),
          ('Paryz', 'Francja', 5, 5)]

In [82]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [83]:
conn.executemany(stmt, data)

In [84]:
conn.commit()

### Czytanie danych z bazy danych

Większość pythonowych sterowników SQL-a wykonując operację wyboru danych z tabeli, zwraca listę krotek:

In [85]:
cursor = conn.execute('select * from test')

In [86]:
rows = cursor.fetchall()

In [87]:
rows

[('Warszawa', 'Polska', 2.0, 6),
 ('Berlin', 'Niemcy', 4.0, 3),
 ('Paryz', 'Francja', 5.0, 5)]

## Łączenie zbiorów danych

Dane znajdujące się w obiektach pandas mogą być ze sobą łączone na wiele sposobów:

• Funkcja `pandas.merge` łączy wiersze ramek danych na podstawie jednego lub kilku kluczy. Działanie tej funkcji wyda się znajome użytkownikom baz SQL i innych relacyjnych baz danych.
Jest ona implementacją bazodanowej operacji łączenia join.

• Funkcja `pandas.concat` łączy (spina) obiekty wzdłuż wybranych osi.

**Więcej w ksiązce "Python w analizie danych" Wes McKinney**

## Łączenie ramek danych w stylu łączenia elementów baz danych
Operacje łączenia merge i join łączą zbiory danych, tworząc połączenia pomiędzy wierszami za pomocą jednego lub kilku kluczy.

Operacje takie są najważniejszymi elementami procesu obsługi relacyjnych baz danych (np. baz opartych na SQL-u).

Funkcja pandas.merge jest punktem wejścia
umożliwiającym stosowanie takich algorytmów na zbiorze danych.

In [88]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [89]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [90]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


Oto przykład operacji łączenia typu wiele do jednego.

Dane obiektu df1 składają się z wielu wierszy oznaczonych etykietami a i b, a obiekt df2 zawiera tylko jeden wiersz dla każdej wartości w kolumnie key.

W wyniku wywołania funkcji pandas.merge i połączenia tych obiektów uzyskamy:

In [91]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


Zauważ, że nie określiłem kolumny, dla której ma być wykonywana operacja łączenia.

Jeżeli taka informacja nie zostanie jawnie podana, to funkcja pandas.merge używa pokrywających się nazw kolumn jako kluczy.

Pomimo tego automatycznego mechanizmu jawne definiowanie kolumny jest
uważane za dobrą praktykę:

In [92]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


## Przykład praktyczny

In [93]:
# Dane o krajach
kraje = pd.DataFrame({
      'kraj_id': [1, 2, 3],
      'kraj': ['Polska', 'Niemcy', 'Francja']
  })

# Dane o emisji CO2
emisja = pd.DataFrame({
      'kraj_id': [1, 2, 3],
      'rok': [2020, 2020, 2020],
      'CO2': [350, 450, 380]
  })

In [94]:
# Łączenie tabel po kraju_id
wynik = pd.merge(kraje, emisja, on='kraj_id')
print(wynik)

   kraj_id     kraj   rok  CO2
0        1   Polska  2020  350
1        2   Niemcy  2020  450
2        3  Francja  2020  380


In [95]:
# Dane z 2020 roku
dane_2020 = pd.DataFrame({
      'kraj': ['Polska', 'Niemcy'],
      'emisja': [350, 450]
})

# Dane z 2021 roku
dane_2021 = pd.DataFrame({
      'kraj': ['Polska', 'Niemcy'],
      'emisja': [340, 440]
})

In [96]:
# Łączenie ramek danych pionowo (jedno pod drugim)
wynik_pionowo = pd.concat([dane_2020, dane_2021], ignore_index=True)
print(wynik_pionowo)

     kraj  emisja
0  Polska     350
1  Niemcy     450
2  Polska     340
3  Niemcy     440


In [97]:
# Dane o emisji CO2
emisja = pd.DataFrame({
    'kraj': ['Polska', 'Niemcy'],
    'CO2': [350, 450]
})

# Dane o konsumpcji energii
energia = pd.DataFrame({
    'kraj': ['Polska', 'Niemcy'],
    'energia': [100, 150]
})

In [98]:
# Łączenie ramek danych poziomo (obok siebie)
wynik_poziomo = pd.concat([emisja, energia], axis=1)
print(wynik_poziomo)

     kraj  CO2    kraj  energia
0  Polska  350  Polska      100
1  Niemcy  450  Niemcy      150
